In [1]:
import requests
import numpy as np
import os
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [2]:
api_key = "ZrQEPSkKYWxleGFuZHJvc2xpYXBhdGVzQGdtYWlsLmNvbQ=="

In [3]:
data = pd.read_csv("/Users/alexanderliapatis/Desktop/Projects/draft_project/data.csv")

In [4]:
vin_list = np.unique(data["vin"].values).tolist()

In [6]:
if os.path.isfile(
    "/Users/alexanderliapatis/Desktop/Projects/draft_project/features_data.csv"
):
    ft_data = pd.read_csv(
        "/Users/alexanderliapatis/Desktop/Projects/draft_project/features_data.csv"
    )
else:
    ft_data = pd.DataFrame(columns=["vin"])

ft_data.shape

(5608, 62)

In [6]:
for vin in vin_list[:-10]:
    if vin not in ft_data["vin"].values:
        url = f"https://auto.dev/api/vin/{vin}?apikey={api_key}"

        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=1)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount("http://", adapter)
        session.mount("https://", adapter)

        response = session.get(url)
        data = pd.DataFrame()
        data["vin"] = [vin]
        for item in response.json().keys():
            if item not in ["options", "years", "colors"]:
                if isinstance(response.json()[item], dict):
                    data = pd.concat(
                        [
                            data,
                            pd.json_normalize(response.json()[item]).add_suffix(
                                "_" + item
                            ),
                        ],
                        axis=1,
                    )
                else:
                    data[item] = response.json()[item]

        ft_data = pd.concat([ft_data, data], axis=0)